In [40]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import json
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import lightgbm as lgb
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import logging
from torch.utils.data import DataLoader, TensorDataset

In [41]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bidding_imbalance_price = pd.read_csv('bidding_training_predictions_imbalance_price.csv')
df_bidding_imbalance_price = df_bidding_imbalance_price[["timestamp_utc","imbalance_price_predictions"]]
df_bidding_imbalance_price.timestamp_utc = pd.to_datetime(df_bidding_imbalance_price.timestamp_utc)
df_bidding_day_ahead_price = pd.read_csv('bidding_training_predictions_day_ahead_price.csv')
df_bidding_day_ahead_price = df_bidding_day_ahead_price[["timestamp_utc","day_ahead_price_predictions"]]
df_bidding_day_ahead_price.timestamp_utc = pd.to_datetime(df_bidding_day_ahead_price.timestamp_utc)
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")

In [42]:
df_bbidding.timestamp_utc = pd.to_datetime(df_bbidding.timestamp_utc)
df_bbidding = df_bbidding.merge(df_bidding_imbalance_price, on="timestamp_utc")
df_bbidding = df_bbidding.merge(df_bidding_day_ahead_price, on="timestamp_utc")

In [43]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price']
    Target_MW = row['Target_MW']
    imbalance_price = row['imbalance_price']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_optimal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))



In [44]:
df_bbidding.revenue_optimal.mean()

56634.21239495645

In [45]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['1']
    imbalance_price = row['imbalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade_prediction'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_normal_prediction'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade_prediction'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade_prediction']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade_prediction']))


In [46]:
df_bbidding.revenue_normal_prediction.mean()

33860.38059863207

In [47]:
df_bbidding["trade_residuals"] = df_bbidding["optimized_trade"] - df_bbidding["optimized_trade_prediction"]
y = df_bbidding["trade_residuals"]
X = df_bbidding[["day_ahead_price_predictions","imbalance_price_predictions","optimized_trade_prediction","cos_hour","cos_day","1","9"]]

In [9]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import Dataset, DataLoader
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# import numpy as np

# # Convert data to tensors
# class TradeDataset(Dataset):
#     def __init__(self, X, y):
#         self.X = torch.FloatTensor(X)
#         self.y = torch.FloatTensor(y.values.reshape(-1, 1))
        
#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

# # Define the MLP model
# class MLP(nn.Module):
#     def __init__(self, input_size):
#         super(MLP, self).__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(input_size, 128),
#             nn.SELU(),
#             nn.Dropout(0.2),
#             nn.Linear(128, 256),
#             nn.SELU(),
#             nn.Dropout(0.2),
#             nn.Linear(256, 64),
#             nn.SELU(),
#             nn.Linear(64, 1)
#         )
        
#     def forward(self, x):
#         return self.layers(x)

# # Prepare the data
# scaler1 = StandardScaler()
# X_scaled = scaler1.fit_transform(X)

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# # Create data loaders
# train_dataset = TradeDataset(X_train, y_train)
# test_dataset = TradeDataset(X_test, y_test)

# batch_size = 512
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size)

# # Initialize model, loss function, and optimizer
# model = MLP(input_size=X.shape[1])
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Training loop
# num_epochs = 1000
# train_losses = []
# test_losses = []

# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0
#     for batch_X, batch_y in train_loader:
#         optimizer.zero_grad()
#         outputs = model(batch_X)
#         loss = criterion(outputs, batch_y)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()
    
#     # Calculate average training loss
#     train_loss = train_loss / len(train_loader)
#     train_losses.append(train_loss)
    
#     # Validation
#     model.eval()
#     test_loss = 0
#     with torch.no_grad():
#         for batch_X, batch_y in test_loader:
#             outputs = model(batch_X)
#             loss = criterion(outputs, batch_y)
#             test_loss += loss.item()
    
#     test_loss = test_loss / len(test_loader)
#     test_losses.append(test_loss)
    
#     if (epoch + 1) % 10 == 0:
#         print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

# # Plot training curves
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 6))
# plt.plot(train_losses, label='Training Loss')
# plt.plot(test_losses, label='Test Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Test Loss over Time')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Evaluate final model
# model.eval()
# with torch.no_grad():
#     X_test_tensor = torch.FloatTensor(X_test)
#     y_pred = model(X_test_tensor).numpy()
    
# # Calculate metrics
# from sklearn.metrics import mean_squared_error, r2_score
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'\nFinal Results:')
# print(f'MSE: {mse:.4f}')
# print(f'RMSE: {np.sqrt(mse):.4f}')
# print(f'R2 Score: {r2:.4f}')

# # Plot predictions vs actual values
# plt.figure(figsize=(10, 6))
# plt.scatter(y_test, y_pred, alpha=0.5)
# plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
# plt.xlabel('Actual Values')
# plt.ylabel('Predicted Values')
# plt.title('Predicted vs Actual Values')
# plt.grid(True)
# plt.show()

In [10]:
# torch.save(model.state_dict(), 'trade_residuals_model.pth')

In [11]:


# logging.basicConfig(level=logging.INFO)

# def objective(trial, X_train, X_test, y_train, y_test):
#     """
#     Objective function for Optuna optimization
#     """
#     params = {
#         'objective': 'regression',
#         'metric': 'l2',
#         'boosting_type': 'gbdt',
#         'verbose': -1,
        
#         # Parameters to optimize
#         'num_leaves': trial.suggest_int('num_leaves', 20, 300),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         'max_depth': trial.suggest_int('max_depth', 3, 20),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'subsample': trial.suggest_float('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
#         'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
#         'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
#         'min_child_weight': trial.suggest_float('min_child_weight', 0.001, 10.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10)
#     }

#     # Create datasets
#     train_data = lgb.Dataset(X_train, y_train)
#     valid_data = lgb.Dataset(X_test, y_test, reference=train_data)

#     # Train model with callbacks
#     model = lgb.train(
#         params,
#         train_data,
#         valid_sets=[valid_data],
#         num_boost_round=1000,
#         callbacks=[
#             lgb.early_stopping(stopping_rounds=20),
#             lgb.log_evaluation(period=100)
#         ]
#     )

#     # Predict and calculate loss
#     y_pred = model.predict(X_test)
#     loss = mean_squared_error(y_test, y_pred)

#     # Log progress
#     logging.info(f"Trial {trial.number} - MSE: {loss:.4f}")

#     return loss

# # Split data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create and optimize study
# study = optuna.create_study(direction="minimize")
# study.optimize(
#     lambda trial: objective(trial, X_train, X_test, y_train, y_test),
#     n_trials=20
# )

# # Get best parameters and train final model
# best_params = study.best_params
# best_params['objective'] = 'regression'
# best_params['metric'] = 'l2'
# best_params['boosting_type'] = 'gbdt'
# best_params['verbose'] = -1

# print("\nBest parameters:")
# for key, value in best_params.items():
#     print(f"    {key}: {value}")

# # Train final model with best parameters
# train_data = lgb.Dataset(X_train, y_train)
# valid_data = lgb.Dataset(X_test, y_test, reference=train_data)

# final_model = lgb.train(
#     best_params,
#     train_data,
#     valid_sets=[valid_data],
#     num_boost_round=1000,
#     callbacks=[
#         lgb.early_stopping(stopping_rounds=20),
#         lgb.log_evaluation(period=100)
#     ]
# )

# # Make predictions and evaluate
# y_pred = final_model.predict(X_test)
# final_mse = mean_squared_error(y_test, y_pred)
# print(f"\nFinal MSE: {final_mse:.4f}")

# # Calculate feature importance
# importance = final_model.feature_importance(importance_type='gain')
# feature_names = final_model.feature_name()

# feature_importance = pd.DataFrame({
#     'Feature': feature_names,
#     'Importance': importance
# }).sort_values('Importance', ascending=False)

# print("\nTop 20 Most Important Features:")
# print(feature_importance.head(20))

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Create dataset for LightGBM
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
import joblib


# Define parameters
params = {
    'objective': 'regression',
    'metric': 'l2',
    'boosting_type': 'gbdt',
    'num_leaves': 237,
    'learning_rate': 0.017557987676990648,
    'max_depth': 13,
    'min_child_samples': 5,
    'subsample': 0.9285594584090331,
    'colsample_bytree': 0.9163968558386822,
    'reg_alpha': 0.17689372588678054,
    'reg_lambda': 0.7918520649641817,
    'min_split_gain': 0.3740091850958246,
    'min_child_weight': 8.558715178577113,
    'bagging_freq': 6,
    'verbose': -1
}

# Train the model with early stopping
gbm = lgb.train(params,
                train_set=lgb_train,
                num_boost_round=1000,  # Set high to enable early stopping
                valid_sets=[lgb_eval],  # Use test set for early stopping validation
                callbacks=[lgb.early_stopping(stopping_rounds=10)])  # Early stopping callback

# Predict on test set
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Save the model using joblib
joblib.dump(gbm, 'lightgbm_model.joblib')
print("Model saved as lightgbm_model.joblib")


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[788]	valid_0's l2: 117729
Mean Squared Error: 117729.0149270751
Model saved as lightgbm_model.joblib


In [50]:
predictions1 = gbm.predict(X)
df_bbidding["trade_residuals_predictions"] = predictions1 + df_bbidding["optimized_trade_prediction"]

In [51]:
df_bbidding['revenue_normal_prediction'] = df_bbidding['day_ahead_price'] * df_bbidding['trade_residuals_predictions'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['trade_residuals_predictions']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['trade_residuals_predictions']))

In [52]:
df_bbidding.revenue_normal_prediction.mean()

48261.6212235938

In [53]:
import torch
import torch.nn as nn

class LSTMPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.1):
        super(LSTMPredictor, self).__init__()
        
        # Parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.dropout = dropout

        # Define the LSTM layer(s)
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, 
                            num_layers=self.num_layers, batch_first=True, dropout=self.dropout)
        
        # Fully connected layer to map LSTM output to the target size
        self.fc = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, x):
        # Initialize hidden and cell states for LSTM
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Cell state

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # We only need the output
        
        # Get the last output (many-to-one), out[:, -1, :] gives the last time step
        out = out[:, -1, :]
        
        # Pass the output through a fully connected layer
        out = self.fc(out)
        
        return out


In [54]:
current_dir = os.getcwd()

path_df = os.path.abspath(os.path.join(current_dir, '..', 'basic_files'))
df_total_solar = pd.read_csv(os.path.join(path_df, 'solar_total_production.csv'))
df_total_solar.generation_mw = df_total_solar.generation_mw *0.5
df_total_wind = pd.read_csv(os.path.join(path_df, 'wind_total_production.csv'))
df_total_wind.generation_mw = df_total_wind.generation_mw *0.5 - df_total_wind.boa
df_imbalance_price = pd.read_csv(os.path.join(path_df, 'imbalance_price.csv'))
df_day_ahead_price = pd.read_csv(os.path.join(path_df, 'day_ahead_price.csv'))
df_market_price = pd.read_csv(os.path.join(path_df, 'market_index.csv'))

# Get the path to the 'logs' directory in the parent directory
path = os.path.abspath(os.path.join(current_dir, '..', 'logs'))
files = os.listdir(path)
txt_files = [file for file in files if file.endswith('.txt')]
data = []
for file in txt_files:
    with open(os.path.join(path, file), 'r') as f:
        try:
            json_data = json.load(f)
            data.append(json_data)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from file: {file}")
date_name = []
for i in range(len(data)):
    date_name.append(data[i]["prediction_date"])

In [55]:
# Extrahiere Daten
dataframe_list = []

for entry in data:
    prediction_date = entry['prediction_date']
    
    # Iteriere durch jedes 'submission' Element
    for submission in entry['solution']['submission']:
        timestamp = submission['timestamp']
        probabilistic_forecast = submission['probabilistic_forecast']
        
        # Extrahiere die Werte von 'probabilistic_forecast' und füge sie der Liste hinzu
        row = {
            'prediction_date': prediction_date,
            'timestamp': timestamp,
            '1': probabilistic_forecast.get('10', None),
            '2': probabilistic_forecast.get('20', None),
            '3': probabilistic_forecast.get('30', None),
            '4': probabilistic_forecast.get('40', None),
            '5': probabilistic_forecast.get('50', None),
            '6': probabilistic_forecast.get('60', None),
            '7': probabilistic_forecast.get('70', None),
            '8': probabilistic_forecast.get('80', None),
            '9': probabilistic_forecast.get('90', None)
        }
        dataframe_list.append(row)

# Erstelle DataFrame
df_api_new = pd.DataFrame(dataframe_list)


In [56]:
df_api_new = df_api_new.groupby("timestamp").last().reset_index()
df_api_new.timestamp = pd.to_datetime(df_api_new.timestamp)

In [57]:

# Create a continuous time series from the minimum to maximum timestamp at 30-minute intervals
full_timestamp_range = pd.date_range(start=df_api_new['timestamp'].min(), end=df_api_new['timestamp'].max(), freq='30min')
# Reindex the dataframe using the full range of timestamps
df_api_new_1 = df_api_new.set_index('timestamp').reindex(full_timestamp_range, method=None)
df_api_new_1 = df_api_new_1.reset_index().rename(columns={'index': 'timestamp'})
# Create the 'prediction_date' column based on the timestamp
df_api_new_1['prediction_date'] = df_api_new_1['timestamp'].dt.date

In [58]:
df_day_ahead_price.timestamp_utc = pd.to_datetime(df_day_ahead_price.timestamp_utc)
df_market_price.timestamp_utc = pd.to_datetime(df_market_price.timestamp_utc)
df_imbalance_price.timestamp_utc = pd.to_datetime(df_imbalance_price.timestamp_utc)
df_api_new_merged = pd.merge(df_api_new_1,df_day_ahead_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_market_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged = pd.merge(df_api_new_merged,df_imbalance_price, left_on='timestamp', right_on='timestamp_utc', how='left')
df_api_new_merged["day_ahead_price"] = df_api_new_merged["price_x"].rename("day_ahead_price")
df_api_new_merged["market_price"] = df_api_new_merged["price_y"].rename("market_price")
df_api_new_merged["settlement_period"] = df_api_new_merged["settlement_period_x"].rename("settlement_period")
df_api_new_merged["cos_hour"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.hour/24)
df_api_new_merged["cos_day"] = np.cos(2*np.pi*df_api_new_merged["timestamp"].dt.day/7)


In [59]:
df_api_new_merged1 = df_api_new_merged[["timestamp_utc","market_price","day_ahead_price","volume","settlement_period","cos_hour","cos_day","1","2","3","4","5","6","7","8","9","imbalance_price"]].copy()
df_api_new_merged1.loc[:,"market_price_lag96h"] = df_api_new_merged1["market_price"].shift(192)
df_api_new_merged1.loc[:,"imbalance_price_lag96h"] = df_api_new_merged1["imbalance_price"].shift(192)
df_api_new_merged1.loc[:,"day_ahead_price_lag1week"] = df_api_new_merged1["day_ahead_price"].shift(192)
df_api_new_merged1.loc[:,"volume_lag96h"] = df_api_new_merged1["volume"].shift(96)
df_api_new_merged1.dropna(inplace=True)
df_api_new_merged1 = df_api_new_merged1.groupby("timestamp_utc").last().reset_index()


In [60]:
solar_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv')
wind_total = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/wind_total_production.csv')
solar_total.generation_mw = solar_total.generation_mw * 0.5
wind_total.generation_mw = wind_total.generation_mw * 0.5 - wind_total.boa
solar_total.timestamp_utc = pd.to_datetime(solar_total.timestamp_utc)
wind_total.timestamp_utc = pd.to_datetime(wind_total.timestamp_utc)

In [61]:
df_api_new_merged2 = pd.merge(df_api_new_merged1,solar_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = pd.merge(df_api_new_merged2,wind_total, on="timestamp_utc", how="inner")
df_api_new_merged2 = df_api_new_merged2.rename(columns={
    "generation_mw_x": "generation_solar",
    "generation_mw_y": "generation_wind"
})
df_api_new_merged2 = df_api_new_merged2.groupby("timestamp_utc").last().reset_index()


In [62]:
import pickle
df_api_new_merged2_X = df_api_new_merged2[["market_price_lag96h","imbalance_price_lag96h","day_ahead_price_lag1week","volume_lag96h",
                    "cos_hour","cos_day","1","2","3","4","5","6","7","8","9"]].copy()

scaler_path = "LSTM_imbalance_scaler.pkl"
# Laden des StandardScalers aus der Datei
with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

# Skalieren der Daten
df_api_new_merged2_X_scaled = scaler.transform(df_api_new_merged2_X)

# Konvertieren der Daten in PyTorch-Tensoren
X_test = torch.tensor(df_api_new_merged2_X_scaled, dtype=torch.float32)
X_test = X_test.unsqueeze(1)  # Adds a sequence length dimension

c:\Users\paulh\anaconda3\envs\HEFTcom24\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [63]:
input_size = 15  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_imbalance_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["imvalance_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_6324\1576695596.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_imbalance_pr

In [64]:
input_size = 15  # Number of features
hidden_size = 64              # Number of LSTM units
num_layers = 3                 # Number of LSTM layers
output_size = 1                # Always 9 for 9 quantiles
dropout = 0.1  
model_imbalance = LSTMPredictor(input_size, hidden_size, num_layers, output_size, dropout=dropout)
model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_price.pth"))
# Modell in den Evaluierungsmodus versetzen
model_imbalance.eval()
with torch.no_grad():
    predictions = model_imbalance(X_test)
predictions = predictions.numpy()
df_api_new_merged2["day_ahead_price_predictions"] = predictions

C:\Users\paulh\AppData\Local\Temp\ipykernel_6324\823899303.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_imbalance.load_state_dict(torch.load("LSTM_day_ahead_pri

In [65]:
df_api_new_merged2["Total_MW"] = df_api_new_merged2["generation_solar"] + df_api_new_merged2["generation_wind"]

In [66]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, DAP, Target_MW, imbalance_price):
    return zb * DAP + (Target_MW - zb) * (imbalance_price - 0.07 * (Target_MW - zb))

# Negative revenue function (for minimization)
def negative_revenue(zb, DAP, Target_MW, imbalance_price):
    return -revenue(zb, DAP, Target_MW, imbalance_price)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    DAP = row['day_ahead_price_predictions']
    Target_MW = row['1']
    imbalance_price = row['imvalance_price_predictions']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(DAP, Target_MW, imbalance_price), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_api_new_merged2['optimized_trade'] = df_api_new_merged2.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_api_new_merged2['revenue_normal'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade']))


In [67]:
df_api_new_merged2.revenue_normal.mean()

21886.00915783455

In [68]:
df_api_new_merged2
X = df_api_new_merged2[["day_ahead_price_predictions","imvalance_price_predictions","optimized_trade","cos_hour","cos_day","1","9"]]
predictions = gbm.predict(X)
df_api_new_merged2["newly_optimized_trade"] = predictions + df_api_new_merged2["optimized_trade"]

In [69]:
df_api_new_merged2['revenue_normal_new'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['newly_optimized_trade'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['newly_optimized_trade']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['newly_optimized_trade']))

In [70]:
df_api_new_merged2.revenue_normal_new.mean()

19745.46583094493

In [35]:
# X_scaled = scaler1.transform(X.values)
# X_scaled = torch.tensor(X_scaled, dtype=torch.float32)

In [36]:
# model.eval()
# with torch.no_grad():
#     predictions_mlp = model(X_scaled)
# predictions_mlp = predictions_mlp.numpy()
# predictions_mlp.shape

In [37]:
# (predictions_mlp + df_api_new_merged2[["optimized_trade"]]).shape

In [38]:
# df_api_new_merged2["optimized_trade_prediction_mlp"] = predictions_mlp + df_api_new_merged2[["optimized_trade"]]

In [39]:
df_api_new_merged2['revenue_normal_new'] = df_api_new_merged2['day_ahead_price'] * df_api_new_merged2['optimized_trade_prediction_mlp'] + \
                         (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade_prediction_mlp']) * \
                         (df_api_new_merged2['imbalance_price'] - 0.07 * (df_api_new_merged2['Total_MW'] - df_api_new_merged2['optimized_trade_prediction_mlp']))

KeyError: 'optimized_trade_prediction_mlp'

In [91]:
df_api_new_merged2.revenue_normal_new.mean()

2047.165563438268